In [22]:
import gc, argparse, sys, os, errno
%pylab inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
sns.set_style('whitegrid')
import h5py
import os
from tqdm import tqdm_notebook as tqdm
import scipy
import sklearn
from scipy.stats import pearsonr
import warnings
warnings.filterwarnings('ignore')
from numba import jit

Populating the interactive namespace from numpy and matplotlib


In [23]:
cd ..

/home/chenxupeng/projects


- [x] implement  https://github.com/Silver-Shen/Causally-Regularized-Learning
- [x] test on training data
- [ ] multi class
- [ ] numba accelarate

In [6]:
def oneHotEncoding(y, numOfClasses):
    """
    Convert a vector into one-hot encoding matrix where that particular column value is 1 and rest 0 for that row.
    :param y: Label vector
    :param numOfClasses: Number of unique labels
    :return: one-hot encoding matrix
    """
    y = np.asarray(y, dtype='int32')
    if len(y) > 1:
        y = y.reshape(-1)
    if not numOfClasses:
        numOfClasses = np.max(y) + 1
    yMatrix = np.zeros((len(y), numOfClasses))
    yMatrix[np.arange(len(y)), y] = 1
    return yMatrix
    

In [7]:
X = 2*np.round(np.random.rand(1000, 20))-1; # 1000 samples and 20 features
beta_true = np.ones([20, 1]);
Y = (sigmoid(np.dot(X,beta_true))>=0.5).astype('double');
lambda0 = 1; #Logistic loss
lambda1 = 0.1; #Balancing loss
lambda2 = 1; #L_2 norm of sample weight
lambda3 = 0; #L_2 norm of beta
lambda4 = 0.001; #L_1 norm of bata
lambda5 = 1; #Normalization of sample weight
MAXITER = 1000;
ABSTOL = 1e-3;
W_init = np.random.rand(1000, 1);
beta_init = 0.5*np.ones([20, 1]);

- [ ] TODO: softmax more complex

In [5]:
#def sigmoid(x):
#    return 1/(1+np.exp(-x))

In [24]:
def sigmoid(x):
    #z = np.dot(X,W)
    exp_x = np.exp(x)
    sum_exp_x = np.sum(exp_x,axis=1)
    sum_exp_x= sum_exp_x.reshape((x.shape[0],1))
    sigmoid = exp_x / sum_exp_x
    return sigmoid

## J_cost
- [ ] TODO: softmax have different lambda0 term

Calculate the loss function without the non-differentiable part

In [9]:
W = W_init
beta = beta_init

In [ ]:
# to implement softmax, change lambda0 term with softmax loss

def softmaxEquation(scores):
    """
    It calculates a softmax probability
    :param scores: A matrix(wt * input sample)
    :return: softmax probability
    """
    scores -= np.max(scores)
    prob = (np.exp(scores).T / np.sum(np.exp(scores), axis=1)).T
    return prob

def computeLoss(x, yMatrix,wt,regStrength):
    """
    It calculates a cross-entropy loss and gradient to update the weights.
    :param x: An input sample
    :param yMatrix: Label as one-hot encoding
    :return:
    """
    numOfSamples = x.shape[0]
    scores = np.dot(x, wt)
    prob = softmaxEquation(scores)

    loss = -np.log(np.max(prob)) * yMatrix
    totalLoss = np.sum(loss) / numOfSamples
    grad = ((-1 / numOfSamples) * np.dot(x.T, (yMatrix - prob))) 
    return totalLoss, grad

In [11]:
lambda0*sum((W*W)*(log(1+exp(X@beta))-Y*(X@beta)))

95.51015297734303

In [33]:
def cross_entropy_loss(W,X,Y,beta):
    '''
    part of J_loss first term
    '''
    n = X.shape[0]
    Y_cap = sigmoid(X@beta)
    loss = (-1/n)*np.sum((W*W)*Y*np.log(Y_cap))
    return loss



In [32]:
#lambda0*sum((W*W)*(np.log(1+np.exp(X@beta))-Y*(X@beta)))
@jit
def J_cost(W,beta,X,Y,lambda0, lambda1, lambda2, lambda3, lambda5):
    return lambda0*cross_entropy_loss(W,X,Y,beta) \
         +lambda1*sum(balance_cost(W,X)) \
         +lambda2*((W*W).T@(W*W)) \
         +lambda3*sum(beta**2) \
         +lambda5*(sum(W*W)-1)**2

## balance cost
- [ ] TODO: it seems do not need to change, means than we can adapt this function into any gradient descent problem

In [270]:
@jit
def balance_cost(W=None,X=None,*args,**kwargs):
    m = X.shape[1]
    f_x=np.zeros([m,1])
    for i in np.arange(0,m):
        X_sub=np.copy(X)
        X_sub[:,i]=0
        I=(X[:,i] > 0).astype('double')+eps
        loss=( np.dot( X_sub.T, np.multiply( np.multiply(W,W),I.reshape(-1,1) ) ) ) / (np.dot((np.multiply(W,W)).T,I.reshape(-1,1)))\
            -(np.dot(X_sub.T,(np.multiply((np.multiply(W,W)),(1 - I.reshape(-1,1)))))) / (np.dot((np.multiply(W,W)).T,(1 - I.reshape(-1,1))))
        #print (loss.shape)
        f_x[i]=np.dot(loss.T,loss)
    return f_x


In [271]:
balance_cost(W,X)

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

## balance_grad.m

In [241]:
@jit
def balance_grad(W=None,X=None,*args,**kwargs):
    n,m=X.shape

    g_w=np.zeros([n,m])
    for i in range(0,m):
        X_sub = np.copy(X)
        X_sub[:,i] = 0 # the ith column is treatment
        I = (X[:,i]>0).reshape(-1,1).astype('double')+eps
        J1 = (X_sub.T@((W*W)*I.reshape(-1,1)))/((W*W).T@(I.reshape(-1,1))) \
            -(X_sub.T@((W*W)*(1-I).reshape(-1,1)))/((W*W).T@(1-I).reshape(-1,1))
        dJ1W = 2*(X_sub.T*((W*I)@np.ones([1,m])).T*((W*W).T@I) \
                  -(X_sub.T@(((W*W)*I)@(W*I).T)))/((W*W).T@I)**2 \
                  -2*(X_sub.T*((W*(1-I))@np.ones([1,m])).T*((W*W).T@(1-I)) \
                  -((X_sub.T@( (W*W) * (1-I) )) @  (W*(1-I) ).T ))/((W*W).T@(1-I))**2
        g_w[:,i] = (2 * dJ1W.T @ J1).ravel()

    return g_w

In [242]:
balance_grad(W,X)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## prox_l1.m

- [ ] TODO: it is also universal?

In [356]:
@jit
def prox_l1(v=None,lambda_=None,*args,**kwargs):
    x=np.fmax(0,v - lambda_) - np.fmax(0,- v - lambda_)
    return x

## main function

- [ ] TODO: careful change

In [ ]:
def cross_entropy_loss(W,X,Y,beta):
    '''
    part of J_loss first term
    '''
    n = X.shape[0]
    Y_cap = sigmoid(X@beta)
    loss = (-1/n)*np.sum((W*W)*Y*np.log(Y_cap))
    return loss

def grad_CE(W,X,Y,Y_cap):
    '''
    part of J_loss's first term grad
    '''
    n = X.shape[0]
    grad = (1/n)*np.matmul(X.T,(Y_cap-Y))
    return grad

- [ ] <span class="burk">TODO</span>: 
grad_beta grad_W要改！要根据grad_CE，对比cross_entropy_loss和原loss改

In [ ]:
n = X.shape[0]
def sigmoid(x):
    exp_x = np.exp(x)
    sum_exp_x = np.sum(exp_x,axis=1)
    sum_exp_x= sum_exp_x.reshape((x.shape[0],1))
    sigmoid = exp_x / sum_exp_x
    return sigmoid

lambda0*(-1/n)*np.sum((W*W)*Y*np.log(sigmoid(X@beta))) #NEW ??对比logistic，应该是对的交叉熵


lambda0*sum((W*W)*(np.log(1+np.exp(X@beta))-Y*(X@beta))) #Original

In [ ]:
grad_beta = lambda0*(((sigmoid(X@beta)-Y)*(W*W)).T@X).T  #Original  手推一下 X^T(y-p)

grad_W = 2*lambda0*(np.log(1+np.exp(X@beta))-Y*(X@beta))*W  #Original

In [370]:

@jit
def mainFunc(X, Y, \
    lambda0, lambda1, lambda2, lambda3, lambda4, lambda5,\
    MAXITER, ABSTOL, W_init, beta_init,paras_save_path=None):

    n,m = X.shape
    W = W_init
    W_prev = np.copy(W)
    beta = beta_init
    beta_prev = np.copy(beta)

    parameter_iter = 0.5
    J_loss = np.ones([MAXITER, 1])*(-1)

    lambda_W = 1
    lambda_beta = 1

    W_All = np.zeros([n, MAXITER])
    beta_All = np.zeros([m, MAXITER])


    # Optimization with gradient descent
    for iter in tqdm(range(1,MAXITER+1)):
        # Update beta
        y = np.copy(beta)
        beta = beta + (iter/(iter+3))*(beta-beta_prev) # fast proximal gradient
        f_base = J_cost(W, beta, X, Y, lambda0, lambda1, lambda2, lambda3, lambda5)
        grad_beta = lambda0*(((sigmoid(X@beta)-Y)*(W*W)).T@X).T \
                   +2*lambda3*beta

        while 1:
            z = prox_l1(beta - lambda_beta*grad_beta, lambda_beta*lambda4)
            if J_cost(W, z, X, Y, lambda0, lambda1, lambda2, lambda3, lambda5)\
               <= f_base + grad_beta.T@(z-beta)\
               + (1/(2*lambda_beta))*sum((z-beta)**2):
                break
            lambda_beta = parameter_iter*lambda_beta
            if lambda_beta<eps*eps*eps:
                break
        beta_prev = y
        beta = z

        # Update W
        y = np.copy(W)
        W = W+(iter/(iter+3))*(W-W_prev)
        f_base = J_cost(W, beta, X, Y, lambda0, lambda1, lambda2, lambda3, lambda5)

        grad_W = 2*lambda0*(np.log(1+np.exp(X@beta))-Y*(X@beta))*W \
                +lambda1*balance_grad(W, X)@np.ones([m,1]) \
                +4*lambda2*W*W*W \
                +4*lambda5*(sum(W*W)-1)*W

        while 1:
            z = prox_l1(W-lambda_W*grad_W, 0)
            #print (J_cost(z, beta, X, Y, lambda0, lambda1, lambda2, lambda3, lambda5),
            #f_base , grad_W.T@(z-W),(1/(2*lambda_W))*sum((z-W)**2))
            if J_cost(z, beta, X, Y, lambda0, lambda1, lambda2, lambda3, lambda5)\
                    <= f_base + grad_W.T@(z-W)\
                    + (1/(2*lambda_W+10e-100))*sum((z-W)**2):
                break
            lambda_W = parameter_iter*lambda_W
            if lambda_W<eps*eps*eps:
                break

        W_prev = y
        W = z

        W_All[:,iter-1] = W.ravel()
        beta_All[:,iter-1] = beta.ravel()

        J_loss[iter-1] = J_cost(W, beta, X, Y,\
                              lambda0, lambda1, lambda2, lambda3, lambda5)\
                     + lambda4*sum(abs(beta))
        print (J_loss[iter-1] , J_loss[iter-2])
        if (paras_save_path is not None) & (iter%10==1):
            #'output/models/crlr/somedir'
            if not os.path.exists(paras_save_path):
                os.makedirs(paras_save_path)
            np.savetxt(paras_save_path+'/beta.txt',beta)
            np.savetxt(paras_save_path+'/W.txt',W)
            np.savetxt(paras_save_path+'/J_loss.txt',J_loss)
        if (iter > 1) & ( abs(J_loss[iter-1] - J_loss[iter-2])[0]  < ABSTOL) or (iter == MAXITER):
            break
    W = W*W

    return W, beta, J_loss

HBox(children=(IntProgress(value=0, max=1000), HTML(value='')))

1.0

In [418]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, recall_score, precision_score, \
    roc_curve, precision_recall_curve, average_precision_score, matthews_corrcoef, confusion_matrix

def report_metrics(y_test, y_pred):
    scorers = {'accuracy': accuracy_score,
           'recall': recall_score,
           'precision': precision_score,
           'f1': f1_score,
           'mcc': matthews_corrcoef
    }
    for metric in scorers.keys():
        print('{} = {}'.format(metric, scorers[metric](y_test, y_pred)))


In [419]:
X = 2*np.round(np.random.rand(1000, 20))-1; # 1000 samples and 20 features
beta_true = np.ones([20, 1]);
Y = (sigmoid(np.dot(X,beta_true))>=0.5).astype('double');
lambda0 = 1; #Logistic loss
lambda1 = 0.1; #Balancing loss
lambda2 = 1; #L_2 norm of sample weight
lambda3 = 0; #L_2 norm of beta
lambda4 = 0.001; #L_1 norm of bata
lambda5 = 1; #Normalization of sample weight
MAXITER = 1000;
ABSTOL = 1e-3;

In [420]:
X_train, X_test, y_train, y_test = train_test_split(X,Y.astype('int'))
print('number of training samples: {}, test samples: {}'.format(X_train.shape[0], X_test.shape[0]))

number of training samples: 750, test samples: 250


In [425]:
model = LogisticRegression()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
report_metrics(y_test, y_pred)

In [427]:
W_init = np.random.rand(X_train.shape[0], 1);
beta_init = 0.5*np.ones([20, 1]);

W, beta, J_loss = mainFunc(X_train, y_train,\
        lambda0, lambda1, lambda2, lambda3, lambda4, lambda5,\
        1000, ABSTOL, W_init, beta_init)

HBox(children=(IntProgress(value=0, max=1000), HTML(value='')))

In [428]:
y_pred = (sigmoid(np.dot(X_test,beta))>=0.5).astype('int')
report_metrics(y_test, y_pred)

accuracy = 1.0
recall = 1.0
precision = 1.0
f1 = 1.0
mcc = 1.0
